In [4]:
import pandas as pd

# 1. Define the filename (make sure you renamed it!)
file_path = 'crsp_stock_returns_1985_2024.csv' # Or the new name if you renamed it

# 2. Read only the first 5 rows to see the columns
df_preview = pd.read_csv(file_path, nrows=5)

# 3. Show the columns and the first few rows
print("Columns in the file:")
print(df_preview.columns.tolist())

print("\nFirst 5 rows of data:")
display(df_preview)

Columns in the file:
['PERMNO', 'date', 'NAMEENDT', 'SHRCD', 'EXCHCD', 'SICCD', 'NCUSIP', 'TICKER', 'COMNAM', 'SHRCLS', 'TSYMBOL', 'NAICS', 'PRIMEXCH', 'TRDSTAT', 'SECSTAT', 'PERMCO', 'ISSUNO', 'HEXCD', 'HSICCD', 'CUSIP', 'DCLRDT', 'DLAMT', 'DLPDT', 'DLSTCD', 'NEXTDT', 'PAYDT', 'RCRDDT', 'SHRFLG', 'HSICMG', 'HSICIG', 'DISTCD', 'DIVAMT', 'FACPR', 'FACSHR', 'ACPERM', 'ACCOMP', 'SHRENDDT', 'NWPERM', 'DLRETX', 'DLPRC', 'DLRET', 'TRTSCD', 'NMSIND', 'MMCNT', 'NSDINX', 'BIDLO', 'ASKHI', 'PRC', 'VOL', 'RET', 'BID', 'ASK', 'SHROUT', 'CFACPR', 'CFACSHR', 'ALTPRC', 'SPREAD', 'ALTPRCDT', 'RETX', 'vwretd', 'vwretx', 'ewretd', 'ewretx', 'sprtrn']

First 5 rows of data:


,PERMNO,date,NAMEENDT,SHRCD,EXCHCD,SICCD,NCUSIP,TICKER,COMNAM,SHRCLS,...,CFACSHR,ALTPRC,SPREAD,ALTPRCDT,RETX,vwretd,vwretx,ewretd,ewretx,sprtrn
0,10000,1985-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-2.5625,NaN,1986-01-07,NaN,0.043061,0.040080,0.028021,0.026355,0.045061
1,10000,1986-01-31,1986-12-03,10.0,3.0,3990.0,68391610.0,OMFGA,OPTIMUM MANUFACTURING INC,A,...,1.0,-4.3750,0.250,1986-01-31,C,0.009830,0.008007,0.044071,0.043082,0.002367
2,10000,1986-02-28,NaN,10.0,3.0,3990.0,68391610.0,OMFGA,OPTIMUM MANUFACTURING INC,A,...,1.0,-3.2500,0.250,1986-02-28,-0.257143,0.072501,0.068191,0.060381,0.058938,0.071489
3,10000,1986-03-31,NaN,10.0,3.0,3990.0,68391610.0,OMFGA,OPTIMUM MANUFACTURING INC,A,...,1.0,-4.4375,0.125,1986-03-31,0.365385,0.053887,0.051362,0.047192,0.045679,0.052794
4,10000,1986-04-30,NaN,10.0,3.0,3990.0,68391610.0,OMFGA,OPTIMUM MANUFACTURING INC,A,...,1.0,-4.0000,0.250,1986-04-30,-0.098592,-0.007903,-0.009634,0.016140,0.015141,-0.014148


In [6]:
import numpy as np
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
from pandas.tseries.offsets import MonthEnd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [10]:
# data cleaning things..
raw = pd.read_excel("data_gpr_export.xls")
df = raw[["month", "GPR"]].copy()
df["month"] = pd.to_datetime(df["month"], dayfirst=True, errors="coerce")
df = df.dropna(subset=["month", "GPR"]).sort_values("month").reset_index(drop=True)

# Make month the index 
df = df.set_index("month")
# definition simple difference shock: GPR_t - GPR_{t-1}
# here I just include a simple difference, to compare the AR to something
df["gpr_change"] = df["GPR"].diff()

# 2) AR(1): GPR_t = c + phi * GPR_{t-1} + u_t
gpr = df["GPR"]
gpr_lag = gpr.shift(1)

df["gpr_ar1_pred"] = np.nan
df["gpr_news_shock"] = np.nan


# Start once we have enough data to estimate, for the AR(1)
# Keep in mind we do not want to use future data to hedge ofc,
# Do we then use an expanding or moving window? Feel free to leave suggestions, gang
min_obs = 24 

for i in range(min_obs, len(df)):
    # Use data up to t-1 to fit AR(1)
    y_train = gpr.iloc[1:i]              
    x_train = gpr_lag.iloc[1:i]          
    train = pd.concat([y_train, x_train], axis=1).dropna()

    y = train.iloc[:, 0]
    x = sm.add_constant(train.iloc[:, 1])

    model = sm.OLS(y, x).fit()

    # One-step-ahead prediction for time t
    x_t = sm.add_constant(pd.Series([gpr_lag.iloc[i]], index=["x"]), has_constant="add")
    # Align names to model params
    x_t = pd.DataFrame({"const": [1.0], train.columns[1]: [gpr_lag.iloc[i]]})

    gpr_hat_t = float(model.predict(x_t)[0])
    shock_t = float(gpr.iloc[i] - gpr_hat_t)

    df.iloc[i, df.columns.get_loc("gpr_ar1_pred")] = gpr_hat_t
    df.iloc[i, df.columns.get_loc("gpr_news_shock")] = shock_t

# Standardize shocks (z-scores)
df["gpr_change_z"] = (df["gpr_change"] - df["gpr_change"].mean()) / df["gpr_change"].std()
df["gpr_news_shock_z"] = (df["gpr_news_shock"] - df["gpr_news_shock"].mean()) / df["gpr_news_shock"].std()

# columns:
# gpr_news_shock   : AR(1) innovation in GPR (raw)
# gpr_news_shock_z : standardized AR(1) GPR shock
# gpr_change      : month-to-month change in GPR
# gpr_change_z    : standardized change in GPR
df

,GPR,gpr_change,gpr_ar1_pred,gpr_news_shock,gpr_change_z,gpr_news_shock_z
month,,,,,,
1985-01-01,102.173378,NaN,NaN,NaN,NaN,NaN
1985-02-01,117.102020,14.928642,NaN,NaN,0.420577,NaN
1985-03-01,124.778152,7.676132,NaN,NaN,0.215553,NaN
1985-04-01,87.929001,-36.849152,NaN,NaN,-1.043155,NaN
1985-05-01,103.262848,15.333847,NaN,NaN,0.432032,NaN
...,...,...,...,...,...,...
2025-08-01,136.759415,1.755157,126.131691,10.627724,0.048170,0.321744
2025-09-01,124.114983,-12.644432,127.436132,-3.321149,-0.358899,-0.084686
2025-10-01,154.425201,30.310219,118.259686,36.165515,0.855407,1.065841


In [12]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import MonthEnd
from pandas_datareader import data as web

# --- 1. Load & Clean Stock Data (The "Pro" Version) ---
# We define a NEW variable 'df_stocks_clean' so we don't mess up original data.
print("Loading and cleaning stock data...")

# Load only necessary columns to save memory and avoid DtypeWarnings
cols_needed = ['permno', 'date', 'ret', 'prc', 'shrout', 'siccd', 'dlret']
file_path = 'crsp_stock_returns_1985_2024.csv' 

df_stocks_clean = pd.read_csv(
    file_path, 
    usecols=lambda x: x.lower() in cols_needed, 
    low_memory=False
)
df_stocks_clean.columns = df_stocks_clean.columns.str.lower()

# Fix Dates: Ensure they are always the last day of the month
df_stocks_clean['date'] = pd.to_datetime(df_stocks_clean['date'], errors='coerce') + MonthEnd(0)

# Force Numeric Types
for col in ['ret', 'dlret', 'prc', 'shrout']:
    df_stocks_clean[col] = pd.to_numeric(df_stocks_clean[col], errors='coerce')

# --- 2. Calculate "True" Returns (Accounting for Delistings) ---
# If a company goes bankrupt, RET might be missing, but DLRET shows -1.0 (-100%).
# Formula: (1+RET) * (1+DLRET) - 1
df_stocks_clean['ret'] = df_stocks_clean['ret'].fillna(0)
df_stocks_clean['dlret'] = df_stocks_clean['dlret'].fillna(0)

df_stocks_clean['ret_adj'] = (1 + df_stocks_clean['ret']) * (1 + df_stocks_clean['dlret']) - 1

# --- 3. Calculate Market Cap (Crucial for Value-Weighted Portfolios) ---
# Market Cap = |Price| * Shares Outstanding
# Note: In CRSP, if a price is an average of Bid/Ask, it has a negative sign. We use abs().
df_stocks_clean['mktcap'] = df_stocks_clean['prc'].abs() * df_stocks_clean['shrout']

# Drop invalid rows (must have Permno, Date, Return, and Market Cap)
df_stocks_clean = df_stocks_clean.dropna(subset=['permno', 'date', 'ret_adj', 'mktcap'])

print(f"Stock data cleaned. Rows: {len(df_stocks_clean)}")

# --- 4. Get Fama-French Factors (Control Variables) ---
print("Downloading Fama-French Factors...")
ff3 = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start='1985', end='2025')[0]
ff3 = ff3.reset_index().rename(columns={'Date': 'date'})
ff3[['Mkt-RF', 'SMB', 'HML', 'RF']] = ff3[['Mkt-RF', 'SMB', 'HML', 'RF']] / 100.0 # Convert % to decimal
ff3['date'] = ff3['date'].dt.to_timestamp(freq='M') + MonthEnd(0)

mom = web.DataReader('F-F_Momentum_Factor', 'famafrench', start='1985', end='2025')[0]
mom = mom.reset_index().rename(columns={'Date': 'date'})
mom[['Mom']] = mom[['Mom']] / 100.0
mom['date'] = mom['date'].dt.to_timestamp(freq='M') + MonthEnd(0)

factors = ff3.merge(mom, on='date', how='inner')

# --- 5. Prepare Your GPR Data for Merge ---
# You already created 'df' in your previous step. We'll just ensure it's merge-ready.
# We create a copy to be safe.
df_gpr_merge = df.copy() 
if 'month' in df_gpr_merge.columns:
    df_gpr_merge = df_gpr_merge.rename(columns={'month': 'date'})
else:
    df_gpr_merge.index.name = 'date'
    df_gpr_merge = df_gpr_merge.reset_index()

df_gpr_merge['date'] = pd.to_datetime(df_gpr_merge['date']) + MonthEnd(0)

# --- 6. MERGE EVERYTHING ---
print("Merging datasets...")
df_master_clean = df_stocks_clean.merge(factors, on='date', how='inner')
df_master_clean = df_master_clean.merge(df_gpr_merge[['date', 'gpr_news_shock_z']], on='date', how='inner')

# Calculate Excess Return (Return - Risk Free Rate)
df_master_clean['excret'] = df_master_clean['ret_adj'] - df_master_clean['RF']

# Drop the early years where Shock is NaN (1985-1987)
df_master_clean = df_master_clean.dropna(subset=['gpr_news_shock_z'])

print("SUCCESS! 'df_master_clean' is ready.")
display(df_master_clean[['date', 'permno', 'ret_adj', 'mktcap', 'gpr_news_shock_z']].head())

Loading and cleaning stock data...
Stock data cleaned. Rows: 3615692


/var/folders/5g/84r8n25n7kd9999t8_99fh9r0000gn/T/ipykernel_55868/2319166873.py:48: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff3 = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start='1985', end='2025')[0]
/var/folders/5g/84r8n25n7kd9999t8_99fh9r0000gn/T/ipykernel_55868/2319166873.py:48: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff3 = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start='1985', end='2025')[0]
/var/folders/5g/84r8n25n7kd9999t8_99fh9r0000gn/T/ipykernel_55868/2319166873.py:53: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object'

Merging datasets...
SUCCESS! 'df_master_clean' is ready.


,date,permno,ret_adj,mktcap,gpr_news_shock_z
12,1987-01-31,10000,-0.212121,1581.53125,-0.351919
13,1987-02-28,10000,0.000000,1581.53125,-0.211489
14,1987-03-31,10000,-0.384615,973.25000,-0.523200
15,1987-04-30,10000,-0.062500,912.44134,0.204003
16,1987-05-31,10000,-0.066667,851.59375,-0.251642


In [15]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS

# --- Step 1: Estimate Firm-Level Sensitivity (Rolling Beta) ---

print("Preparing data for regression...")

# 1. CRITICAL FIX: Sort by Permno and Date
# This ensures that when we roll 60 months back, we are actually looking at the PAST 60 months.
df_master_clean = df_master_clean.sort_values(['permno', 'date'])

# 2. Define the Rolling Function
def rolling_beta_pipeline(group):
    # Constraint: We need at least 36 months of data to get a valid beta
    if len(group) < 60: 
        return pd.Series(np.nan, index=group.index)
    
    # Dependent Variable: Excess Return (ret_adj - RF)
    endog = group['excret']
    
    # Independent Variables: 4 Factors + GPR Shock
    exog_vars = ['Mkt-RF', 'SMB', 'HML', 'Mom', 'gpr_news_shock_z']
    exog = sm.add_constant(group[exog_vars])
    
    # Run Rolling OLS
    # window=60 (5 years), min_nobs=36
    rols = RollingOLS(endog, exog, window=60, min_nobs=36)
    rres = rols.fit()
    
    # Return ONLY the coefficient for 'gpr_news_shock_z'
    return rres.params['gpr_news_shock_z']

print("Starting Rolling Beta estimation. This may take 5-10 minutes...")

# 3. Apply the function
# group_keys=False keeps the original index so we can assign it back easily
df_master_clean['beta_gpr'] = df_master_clean.groupby('permno', group_keys=False).apply(rolling_beta_pipeline)

# 4. Lag the Beta
# We use the beta estimated using data up to month t-1 to make decisions in month t.
df_master_clean['beta_gpr_lag'] = df_master_clean.groupby('permno')['beta_gpr'].shift(1)

# 5. Create Analysis Set
# Drop rows where we don't have a beta yet (the first 5 years for each firm)
df_analysis = df_master_clean.dropna(subset=['beta_gpr_lag']).copy()

print("Step 1 Complete!")
print(f"Rows with valid Betas: {len(df_analysis)}")
display(df_analysis[['date', 'permno', 'beta_gpr', 'beta_gpr_lag']].head())

Preparing data for regression...
Starting Rolling Beta estimation. This may take 5-10 minutes...


/var/folders/5g/84r8n25n7kd9999t8_99fh9r0000gn/T/ipykernel_55868/3247383688.py:39: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_master_clean['beta_gpr'] = df_master_clean.groupby('permno', group_keys=False).apply(rolling_beta_pipeline)


Step 1 Complete!
Rows with valid Betas: 1971707


,date,permno,beta_gpr,beta_gpr_lag
89,1992-01-31,10001,-0.002495,-0.001996
90,1992-02-29,10001,-0.003845,-0.002495
91,1992-03-31,10001,-0.003807,-0.003845
92,1992-04-30,10001,-0.003791,-0.003807
93,1992-05-31,10001,-0.004170,-0.003791


In [17]:
# --- Step 2: Sort Firms and Construct Portfolios (The "Bulletproof" Version) ---

print("Sorting firms into deciles...")

# 1. CRITICAL FIX: Lag Market Cap
# We must weight Feb returns using Jan Market Cap to avoid look-ahead bias.
df_analysis = df_analysis.sort_values(['permno', 'date'])
df_analysis['mktcap_lag'] = df_analysis.groupby('permno')['mktcap'].shift(1)

# Drop the first month for each firm (since we don't have a lagged cap for it)
df_analysis = df_analysis.dropna(subset=['mktcap_lag'])

# 2. Sort into 10 Deciles based on LAGGED Beta
# We use duplicates='drop' to handle rare edge cases where betas are identical
df_analysis['decile'] = df_analysis.groupby('date', group_keys=False).apply(
    lambda g: pd.qcut(g['beta_gpr_lag'], 10, labels=False, duplicates='drop')
)

# 3. Robust Value-Weighted Mean Function
def vw_mean_robust(g):
    # Get weights (Lagged Cap) and Returns
    w = g['mktcap_lag']
    r = g['ret_adj']
    
    # Filter out any missing values or zero weights
    mask = w.notna() & r.notna() & (w > 0)
    w = w[mask]
    r = r[mask]
    
    # Safety: If no valid data, return NaN
    if len(w) == 0 or w.sum() == 0:
        return np.nan
        
    # Weighted Average
    return np.average(r, weights=w)

print("Calculating Value-Weighted Portfolio Returns...")

# 4. Calculate Portfolio Returns
port_rets = df_analysis.groupby(['date', 'decile']).apply(vw_mean_robust).unstack()

# 5. SAFETY CHECK: Ensure we have exactly 10 portfolios
# Sometimes qcut drops a bin if data is sparse. We only keep months with full data.
port_rets = port_rets.dropna()

# Now it is safe to rename columns
port_rets.columns = [f'P{i+1}' for i in range(10)]

# 6. Add Factors Back (for Step 3)
port_rets = port_rets.merge(factors.set_index('date'), left_index=True, right_index=True)

print("Step 2 Complete! Portfolio Returns (Lagged VW) Ready.")
display(port_rets.head())

Sorting firms into deciles...


/var/folders/5g/84r8n25n7kd9999t8_99fh9r0000gn/T/ipykernel_55868/3827097342.py:15: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_analysis['decile'] = df_analysis.groupby('date', group_keys=False).apply(


Calculating Value-Weighted Portfolio Returns...
Step 2 Complete! Portfolio Returns (Lagged VW) Ready.


/var/folders/5g/84r8n25n7kd9999t8_99fh9r0000gn/T/ipykernel_55868/3827097342.py:40: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  port_rets = df_analysis.groupby(['date', 'decile']).apply(vw_mean_robust).unstack()


,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,Mkt-RF,SMB,HML,RF,Mom
date,,,,,,,,,,,,,,,
1990-11-30,0.088466,0.027869,0.090943,0.047636,0.068302,0.114082,0.095337,-0.030822,0.114983,0.010339,0.0635,0.0042,-0.0297,0.0057,-0.0536
1990-12-31,-0.001892,-0.004839,-0.010392,-0.020702,0.028421,0.028929,0.028929,0.011568,-0.018708,-0.022763,0.0246,0.0079,-0.0157,0.0060,0.0025
1991-01-31,0.067079,0.074237,0.087636,0.064246,0.077049,0.022471,0.028421,0.002757,0.107683,0.105686,0.0469,0.0378,-0.0148,0.0052,-0.0647
1991-02-28,0.074948,0.112537,0.110595,0.059761,0.054627,0.081695,0.017847,0.026154,0.062414,-0.003130,0.0719,0.0401,-0.0062,0.0048,-0.0494
1991-03-31,0.059199,0.049555,0.010452,0.032648,0.042471,-0.053408,0.038693,-0.019697,-0.039016,0.040941,0.0266,0.0388,-0.0130,0.0044,0.0271
